## Годзун Егор
### ДЗ №1

#### Устанавливаем нужные библиотеки

In [1]:
!pip install clickhouse-connect -q
!pip install pandas

#### Загрузка данных и миграция

In [2]:
import clickhouse_connect
import pandas as pd


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

client_1 = clickhouse_connect.get_client(
    host='localhost',
    port=8123,
    username='default',
    password='12345',
    database='imdb'
)

client_2 = clickhouse_connect.get_client(
    host='localhost',
    port=8124,
    username='default',
    password='12345',
    database='imdb'
)

CREATE_TABLES_SQL_FILE = 'create_tables.sql'
INSERT_IN_TABLES_SQL_FILE = 'insert_in_tables.sql'
MIGRATION_TABLES_SQL_FILE = 'migration.sql'
LIMIT = 10

In [6]:
def execute_sql_file(client, file_path):
    with open(file_path, 'r') as file:
        sql_queries = file.read().split(';')

    for query in sql_queries:
        query = query.strip()
        if query:
            try:
                client.command(query)
                first_line = query.split('\n')[0]
                print(f"Completed: {first_line}")
            except Exception as e:
                print(f"Error: {e}")

In [7]:
def execute_query(client, query):
    try:
        result_df = client.query_df(query)
        result = client.query(query)

        elapsed_s = int(result.summary.get('elapsed_ns', 0)) / 1_000_000_000
        read_rows = int(result.summary.get('read_rows', 0))
        result_rows = int(result.summary.get('result_rows', 0))

        print(f"Время выполнения: {elapsed_s:.3f} сек")
        print(f"Обработано строк: {read_rows:_}")
        print(f"Возвращено строк: {result_rows:_}")

        return result_df.iloc[:LIMIT]

    except Exception as e:
        print(f"Ошибка при выполнении запроса: {str(e)}")

##### Создаем таблицы 

In [8]:
execute_sql_file(client_1, CREATE_TABLES_SQL_FILE)

Completed: CREATE DATABASE IF NOT EXISTS imdb
Completed: DROP TABLE IF EXISTS imdb.name_basics
Completed: DROP TABLE IF EXISTS imdb.title_basics
Completed: DROP TABLE IF EXISTS imdb.title_akas
Completed: DROP TABLE IF EXISTS imdb.title_crew
Completed: DROP TABLE IF EXISTS imdb.title_episode
Completed: DROP TABLE IF EXISTS imdb.title_principals
Completed: DROP TABLE IF EXISTS imdb.title_ratings
Completed: CREATE TABLE IF NOT EXISTS imdb.name_basics
Completed: CREATE TABLE IF NOT EXISTS imdb.title_basics
Completed: CREATE TABLE IF NOT EXISTS imdb.title_akas
Completed: CREATE TABLE IF NOT EXISTS imdb.title_crew
Completed: CREATE TABLE IF NOT EXISTS imdb.title_episode
Completed: CREATE TABLE IF NOT EXISTS imdb.title_principals
Completed: CREATE TABLE IF NOT EXISTS imdb.title_ratings


##### Наполняем таблицы 

In [9]:
execute_sql_file(client_1, INSERT_IN_TABLES_SQL_FILE)

Completed: INSERT INTO imdb.name_basics
Completed: INSERT INTO imdb.title_basics SELECT
Completed: INSERT INTO imdb.title_akas SELECT
Completed: INSERT INTO imdb.title_crew
Completed: INSERT INTO imdb.title_episode
Completed: INSERT INTO imdb.title_principals
Completed: INSERT INTO imdb.title_ratings


##### Смотрим таблицы

In [10]:
result = client_1.query('SHOW TABLES')
for table in result.result_rows:
    print(table[0])

name_basics
title_akas
title_basics
title_crew
title_episode
title_principals
title_ratings


In [11]:
client_1.query_df('SELECT * FROM imdb.title_episode LIMIT 5')

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0031458,tt32857063,<NA>,<NA>
1,tt0041951,tt0041038,1,9
2,tt0042816,tt0989125,1,17
3,tt0042889,tt0989125,<NA>,<NA>
4,tt0043426,tt0040051,3,42


In [12]:
client_1.query_df('SELECT * FROM imdb.title_episode LIMIT 5')

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0031458,tt32857063,<NA>,<NA>
1,tt0041951,tt0041038,1,9
2,tt0042816,tt0989125,1,17
3,tt0042889,tt0989125,<NA>,<NA>
4,tt0043426,tt0040051,3,42


##### Создаем таблицы в другом ClickHouse

In [13]:
execute_sql_file(client_2, CREATE_TABLES_SQL_FILE)

Completed: CREATE DATABASE IF NOT EXISTS imdb
Completed: DROP TABLE IF EXISTS imdb.name_basics
Completed: DROP TABLE IF EXISTS imdb.title_basics
Completed: DROP TABLE IF EXISTS imdb.title_akas
Completed: DROP TABLE IF EXISTS imdb.title_crew
Completed: DROP TABLE IF EXISTS imdb.title_episode
Completed: DROP TABLE IF EXISTS imdb.title_principals
Completed: DROP TABLE IF EXISTS imdb.title_ratings
Completed: CREATE TABLE IF NOT EXISTS imdb.name_basics
Completed: CREATE TABLE IF NOT EXISTS imdb.title_basics
Completed: CREATE TABLE IF NOT EXISTS imdb.title_akas
Completed: CREATE TABLE IF NOT EXISTS imdb.title_crew
Completed: CREATE TABLE IF NOT EXISTS imdb.title_episode
Completed: CREATE TABLE IF NOT EXISTS imdb.title_principals
Completed: CREATE TABLE IF NOT EXISTS imdb.title_ratings


##### Переносим данные в другой ClickHouse

In [14]:
execute_sql_file(client_2, MIGRATION_TABLES_SQL_FILE)

Completed: INSERT INTO imdb.name_basics
Completed: INSERT INTO imdb.title_basics
Completed: INSERT INTO imdb.title_akas
Completed: INSERT INTO imdb.title_crew
Completed: INSERT INTO imdb.title_episode
Completed: INSERT INTO imdb.title_principals
Completed: INSERT INTO imdb.title_ratings


##### Смотрим таблицы

In [15]:
execute_query(client_2, 'SHOW TABLES')

Время выполнения: 0.004 сек
Обработано строк: 7
Возвращено строк: 7


,name
0,name_basics
1,title_akas
2,title_basics
3,title_crew
4,title_episode
5,title_principals
6,title_ratings


In [16]:
execute_query(client_2, 'SELECT * FROM imdb.title_episode LIMIT 5')

Время выполнения: 0.051 сек
Обработано строк: 10
Возвращено строк: 5


,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0031458,tt32857063,<NA>,<NA>
1,tt0041951,tt0041038,1,9
2,tt0042816,tt0989125,1,17
3,tt0042889,tt0989125,<NA>,<NA>
4,tt0043426,tt0040051,3,42


видим, что данные успешно перенесены

##### Выполняем запросы с замером времени

PREWHERE

Выведем фильмы, вышедшие после 2020 года

In [17]:
execute_query(
    client=client_2,
    query="""
            SELECT tconst, primaryTitle, startYear, titleType
            FROM imdb.title_basics
            PREWHERE startYear > 2020 AND titleType = 'movie'
          """
)


Время выполнения: 0.255 сек
Обработано строк: 11_630_899
Возвращено строк: 88_717


,tconst,primaryTitle,startYear,titleType
0,tt0762118,Moe,2023,movie
1,tt25260562,Unacceptable Views,2022,movie
2,tt25260658,Jackpot Island: Kumanthong Returns,2022,movie
3,tt25260664,Furies,2022,movie
4,tt25264464,Informatsia,2022,movie
5,tt2527138,Finster,2022,movie
6,tt25272982,Once Upon A Time in Philly Part 2,2023,movie
7,tt25273052,The Bear's Den,2021,movie
8,tt25273230,Tales from the Void,2023,movie
9,tt25273434,Famine,2022,movie


LEFT JOIN

Выведем фильмы, у которых больше 1000 оценок, сортируем по убыванию рейтинга

In [19]:
execute_query(
    client=client_2,
    query="""
            SELECT
                b.primaryTitle,
                r.averageRating,
                r.numVotes
            FROM imdb.title_basics b
            LEFT JOIN imdb.title_ratings r ON b.tconst = r.tconst
            WHERE b.titleType = 'movie' AND r.numVotes > 1000
            ORDER BY r.averageRating DESC
          """
)

Время выполнения: 0.258 сек
Обработано строк: 13_195_059
Возвращено строк: 45_554


,primaryTitle,averageRating,numVotes
0,Kousalya Tanaya Ragava,10.0,1144
1,HMM Hug Me More,9.8,1083
2,Kora,9.7,1022
3,The Suspect,9.7,1108
4,Golden Opulence: 500 Years of Luxury in Anatolia,9.5,1616
5,Strawberry Melancholy,9.5,4760
6,Jibon Theke Neya,9.3,2305
7,The Shawshank Redemption,9.3,3038716
8,Mana Iddari Prema Katha,9.3,1508
9,The Strangers' Case,9.3,8354


FULL OUTER JOIN

Выведем фильмы, для которых либо нет рейтинга, но есть название, либо нет названия но есть рейтинг

In [23]:
execute_query(
    client=client_2,
    query="""
            SELECT
                b.tconst,
                b.primaryTitle,
                r.averageRating
            FROM imdb.title_basics b
            FULL OUTER JOIN imdb.title_ratings r ON b.tconst = r.tconst
            WHERE r.averageRating IS NULL
                OR b.primaryTitle IS NULL
          """
)

Время выполнения: 0.066 сек
Обработано строк: 0
Возвращено строк: 0


""


OVER QUALIFY

Выведем фильмы, которые занимают 3-е место по рейтингу в порядке возрастания

In [39]:
execute_query(
    client=client_2,
    query="""
            SELECT
                b.primaryTitle,
                r.averageRating,
                DENSE_RANK() OVER(ORDER BY r.averageRating ASC) AS rating_rank
            FROM imdb.title_basics b
            JOIN imdb.title_ratings r ON b.tconst = r.tconst
            WHERE b.titleType = 'movie'
            QUALIFY rating_rank = 3
            ORDER BY b.primaryTitle
          """
)

Время выполнения: 0.333 сек
Обработано строк: 13_195_059
Возвращено строк: 54


,primaryTitle,averageRating,rating_rank
0,15/07: Break of Dawn,1.2,3
1,212: The Power of Love,1.2,3
2,Aks-e Khosoosi,1.2,3
3,Always Cinema,1.2,3
4,Bagyong Bheverlynn,1.2,3
5,Bi Setare,1.2,3
6,Bible Town,1.2,3
7,Cumali Ceber 2,1.2,3
8,Dance Dance,1.2,3
9,Daniel the Wizard,1.2,3


GROUP BY HAVING

Выведем среднюю длину названий по типу и оставим только те типы, у которых средняя длина названия больше 20 символов

In [41]:
execute_query(
    client=client_2,
    query="""
            SELECT
                titleType,
                AVG(LENGTH(primaryTitle)) AS avg_title_length
            FROM imdb.title_basics
            GROUP BY titleType
            HAVING avg_title_length > 20
            ORDER BY avg_title_length DESC
          """
)

Время выполнения: 0.475 сек
Обработано строк: 11_630_899
Возвращено строк: 6


,titleType,avg_title_length
0,tvSpecial,29.970612
1,video,25.689215
2,tvMovie,22.761693
3,tvShort,22.304364
4,tvEpisode,20.307402
5,videoGame,20.032048


EXCEPT

Выведем идентификаторы фильмов после 2020 года, исключая те, у которых рейтинг меньше 5.0

In [42]:
execute_query(
    client=client_2,
    query="""
            SELECT tconst FROM imdb.title_basics
            WHERE titleType = 'movie' AND startYear > 2020
            EXCEPT
            SELECT tconst FROM imdb.title_ratings
            WHERE averageRating < 5.0
          """
)

Время выполнения: 0.162 сек
Обработано строк: 13_195_059
Возвращено строк: 80_765


,tconst
0,tt0780471
1,tt0781524
2,tt0783848
3,tt13250808
4,tt13253056
5,tt13254032
6,tt13254234
7,tt13254306
8,tt13254534
9,tt13254554


CUBE

Выведем для взрослого контента среднюю продолжительность и количество записей по всем типам и их комбинациям

In [69]:
execute_query(
    client=client_2,
    query="""
            SELECT
                isAdult,
                titleType,
                AVG(runtimeMinutes) AS avg_runtime,
                COUNT() AS count
            FROM imdb.title_basics
            GROUP BY CUBE(isAdult, titleType)
            HAVING isAdult = 1
            ORDER BY isAdult, titleType
          """
)

Время выполнения: 0.313 сек
Обработано строк: 11_630_899
Возвращено строк: 11


,isAdult,titleType,avg_runtime,count
0,1,,78.570369,376574
1,1,movie,78.634230,9177
2,1,short,15.153396,2590
3,1,tvEpisode,28.420098,250626
4,1,tvMiniSeries,48.175141,373
5,1,tvMovie,87.085714,91
6,1,tvSeries,39.947791,2885
7,1,tvShort,18.333333,3
8,1,tvSpecial,86.166667,26
9,1,video,114.086384,110220


FORMAT

![JSONF](./photo/jsonf.png)

![VERTICALF](./photo/verticalf.png)